<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/mrc_LSTM_baseline0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Import Libraries and Read Data

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
project_path = "/content/drive/My Drive/AIML-MRC-Capstone/datasets/Squad2.0/TrainingDataset/"

squad_df = pd.read_csv(project_path+'squad_data_final.csv')
squad_df.drop('Unnamed: 0',axis=1,inplace=True)
squad_df.tail(5)

,title,context,question,id,answer_start,answer,plausible_answer_start,plausible_answer,is_impossible,clean_context,clean_question,clean_answer
130301,Matter,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,5a7e070b70df9f001a875439,NaN,NaN,485.0,matter,True,term matter used throughout physics bewilderin...,physics has broadly agreed on the definition o...,IMPOSSIBLE
130302,Matter,"The term ""matter"" is used throughout physics i...",Who coined the term partonic matter?,5a7e070b70df9f001a87543a,NaN,NaN,327.0,Alfvén,True,term matter used throughout physics bewilderin...,who coined the term partonic matter,IMPOSSIBLE
130303,Matter,"The term ""matter"" is used throughout physics i...",What is another name for anti-matter?,5a7e070b70df9f001a87543b,NaN,NaN,350.0,Gk. common matter,True,term matter used throughout physics bewilderin...,what is another name for antimatter,IMPOSSIBLE
130304,Matter,"The term ""matter"" is used throughout physics i...",Matter usually does not need to be used in con...,5a7e070b70df9f001a87543c,NaN,NaN,529.0,a specifying modifier,True,term matter used throughout physics bewilderin...,matter usually does not need to be used in con...,IMPOSSIBLE
130305,Matter,"The term ""matter"" is used throughout physics i...",What field of study has a variety of unusual c...,5a7e070b70df9f001a87543d,NaN,NaN,37.0,physics,True,term matter used throughout physics bewilderin...,what field of study has a variety of unusual c...,IMPOSSIBLE


# Lets use 2 LSTM network for context and question and use a concat layer to merge. y is the answer 

## Step 1 Tokenize clean_context

In [0]:
context_tokenize = tf.keras.preprocessing.text.Tokenizer()
context_tokenize.fit_on_texts(squad_df['clean_context']) #Fit it on clean_context

In [6]:
#check vocab size of clean_context
len(context_tokenize.word_index)

93529

In [7]:
print(context_tokenize.word_index)

{'also': 1, 'one': 2, 'first': 3, 'new': 4, 'many': 5, 'city': 6, 'states': 7, 'used': 8, 'two': 9, 'may': 10, 'time': 11, 'united': 12, 'state': 13, 'world': 14, 'century': 15, 'would': 16, 'war': 17, 'known': 18, 'years': 19, 'however': 20, 'including': 21, 'use': 22, 'system': 23, 'government': 24, 'early': 25, 'people': 26, 'became': 27, 'since': 28, 'later': 29, 'made': 30, 'well': 31, 'number': 32, 'part': 33, 'often': 34, 'called': 35, 'million': 36, 'national': 37, 'north': 38, 'three': 39, '1': 40, 'power': 41, 'population': 42, 'year': 43, 'university': 44, 'several': 45, 'major': 46, 'american': 47, 'although': 48, 'music': 49, 'south': 50, 'area': 51, 'period': 52, 'british': 53, 'u': 54, 'include': 55, 'large': 56, 'law': 57, 'high': 58, 'within': 59, 'public': 60, 'form': 61, 'school': 62, 'language': 63, 'around': 64, 'began': 65, '2': 66, 'west': 67, 'church': 68, 'among': 69, 'second': 70, 'could': 71, 'led': 72, 'french': 73, 'political': 74, 'found': 75, 'great': 76,

In [0]:
#convert the context text to indexes
context_sequence= context_tokenize.texts_to_sequences(squad_df['clean_context'])


In [16]:
squad_df['clean_context'][3000]

'office buildings shanghais financial district including jin mao tower hong kong new world tower evacuated receptionist tibet hotel chengdu said things calm hotel evacuated guests meanwhile workers ford plant sichuan evacuated 10 minutes chengdu shuangliu international airport shut control tower regional radar control evacuated one silkair flight diverted landed kunming result cathay pacific delayed legs quadruple daily hong kong london route due disruption air traffic services chengdu shuangliu airport reopened later evening may 12 offering limited service airport began used staging area relief operations'

In [0]:
context_sequence[3000]

# Step 2 Find out max sequence length of clean_context

In [17]:
# max length of context
max_context_seq_length= max(len(txt) for txt in context_sequence)
print( max_context_seq_length)

# vocab size of context
context_vocab_size=len(context_tokenize.word_index)

426


# Step 3 Padding the sequence for clean_context

In [0]:
context_input_data= tf.keras.preprocessing.sequence.pad_sequences(context_sequence, maxlen=max_context_seq_length, padding='pre')

# Step 4 Build the LSTM Model for 'Context' and get the c and h values

In [0]:
embedding_size = 50
rnn_units=256


In [0]:
# input layer
context_input=tf.keras.layers.Input(shape=(None,))

# Build Embedding layer
context_embedding=tf.keras.layers.Embedding(context_vocab_size+1, embedding_size)

# Get Embedding Layer output
context_embedding_output=context_embedding(context_input)

#LSTM layer and its output
x, state_h1, state_c1= tf.keras.layers.LSTM(rnn_units,return_state=True)(context_embedding_output)

#build a list to feed for concatenation
context_states= [state_h1, state_c1]

In [27]:
context_embedding_output

<tf.Tensor 'embedding_6/Identity:0' shape=(None, None, 50) dtype=float32>

# Step 5 Tokenize the Questions

In [0]:
questions_tokenize= tf.keras.preprocessing.text.Tokenizer()
questions_tokenize.fit_on_texts(squad_df['clean_question'])

In [0]:
#check vocab size of questions
len(questions_tokenize.word_index)

In [0]:
print(questions_tokenize.word_index)

In [0]:
# convert the questions to indexes
questions_sequence= questions_tokenize.text_to_sequences(squad_df['clean_question'])

In [0]:
squad_df['clean_question'][2000]

In [0]:
questions_sequence[2000]

# Step 6 Find out maximum sequence length of Questions

In [0]:
max_question_seq_length=max(len(txt) for txt in questions_sequence)
print(max_question_seq_length)

# vocab size of questions
questions_vocab_size=len(questions_tokenize.word_index)

# Step 7 Padding the sequences for Questions

In [0]:
question_input

# Step 8 Build the LSTM Model for 'Questions' and get the c and h values